In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('../SourceCode')
import bastien_utils
from shallowAE import ShallowAE
from sparseShallowAE import SparseShallowAE_KL, SparseShallowAE_L1, SparseShallowAE_KL_sum
from nonNegSparseShallowAE import Sparse_NonNeg_ShallowAE_KLsum_AsymDecay
from AsymAE_infoGAN.AsymAE_infoGAN import AsymAEinfoGAN
import datetime
import morphoMaths
import keras
import pandas as pd
keras.__version__
from keras.layers import Input, Flatten, Dense, LeakyReLU, Reshape, ReLU, Conv2D, BatchNormalization
from keras import Model, constraints

Using TensorFlow backend.


In [5]:
FILE_PATH = "../"
x_train, _, x_test, _ = bastien_utils.load_data(FILE_PATH, train=True, test=True, subsetTest=False)

In [3]:
nb_rows = 28
nb_columns = 28
nb_input_channels=1
latent_dim = 100
nb_output_channels = 1

# 1. Simple

In [4]:
input_img = Input(shape=(nb_rows, nb_columns,nb_input_channels))  # adapt this if using `channels_first` image data format
x = Conv2D(64, (4, 4), strides=(2, 2), padding='same')(input_img)
x = LeakyReLU(alpha=0.1)(x)
x = Conv2D(128, (4, 4), strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
x = Flatten()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.1)(x)
encoded = Dense(latent_dim, activation='sigmoid')(x)
encoder = Model(input_img, encoded, name='encoder')

encoded_img = Input(shape=(latent_dim,))  
x = Dense(nb_rows*nb_columns*nb_output_channels)(encoded_img)
x = LeakyReLU(alpha=0.1)(x)
decoded = Reshape((nb_rows,nb_columns,nb_output_channels))(x)
decoder = Model(encoded_img, decoded, name='decoder')
encoded = encoder(input_img)
decoded = decoder(encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [8]:
encoder.input_shape

(None, 28, 28, 1)

In [16]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        1088      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         131200    
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
__________

In [2]:
AE = AsymAEinfoGAN()

In [4]:
AE.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               79184     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 79,184
Trainable params: 79,184
Non-trainable params: 0
_________________________________________________________________


In [6]:
AE.train(x_train, X_val=x_test)

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
 1536/60000 [..............................] - ETA: 5:02 - loss: 0.1628

KeyboardInterrupt: 

In [9]:
from AsymAE_infoGAN.testAsymAE import testDims_AsymAE
